In [1]:
filename = 'O2 Grouper Pipeline - Task List.xlsx'

In [11]:
from openpyxl import load_workbook
from openpyxl.styles import Font
from dateutil import parser
import rfc3339
from datetime import datetime


from tools.lib.rdf_project_creator import RDF
from tools.lib.wbs import WBS




def str_to_rfc(datetime):
    return rfc3339.rfc3339(parser.parse(datetime))



schema = {
    "starting_data_row": 3,
    "columns": {
        "wbs": "A",
        "name": "B",
        "description": "C",
        "dependency": "D",
        "estimated_development_time_designer": "E",
        "estimated_development_time_reviewer": "F",
        "estimated_development_time_pm": "G",
        "estimated_development_time_summary": "H",
        "assigned_developer": "I",
        "actual_development_time_start": "J",
        "actual_development_time_finish": "K",
        "actual_development_time_duration": "L",
        "commit_statistics_insertions": "M",
        "commit_statistics_deletions": "N",
        "code_documentation_time": "O",
        "jira_id": "P",
        "commit_id": "Q",
        "sprint": "R",
        "comment": "S"
    }
}


path = './tools/'
workbook = load_workbook(path + filename)
sheet = workbook['Tasks']



rdf = RDF()

rdf.add_root(filename)
rdf.add_baseline_planned('Plan at ' + datetime.now().strftime("%Y %b %d, %H:%M"))


exc_lst = ['wbs', 'name', 'description', 'actual_development_time_duration', 'actual_development_time_start', 'actual_development_time_finish']

i = schema['starting_data_row']
while sheet[schema['columns']['wbs'] + str(i)].value is not None:

    if WBS.check(sheet[schema['columns']['wbs'] + str(i)].value) or sheet[schema['columns']['name'] + str(i)].value is None:

        project = {
            "id": sheet[schema['columns']['wbs'] + str(i)].value,
            "name": sheet[schema['columns']['name'] + str(i)].value,
            "description": str(sheet[schema['columns']['description'] + str(i)].value).replace('"', '\\\"'),
            "wbs": WBS.normalize(sheet[schema['columns']['wbs'] + str(i)].value),
            "parent_id": WBS.parent(sheet[schema['columns']['wbs'] + str(i)].value)
        }

        if sheet[schema['columns']['actual_development_time_duration'] + str(i)].value is not None:
            project['worktime'] = str(sheet[schema['columns']['actual_development_time_duration'] + str(i)].value) + 'h'

        if sheet[schema['columns']['actual_development_time_start'] + str(i)].value is not None:
            project['start'] = str_to_rfc(sheet[schema['columns']['actual_development_time_start'] + str(i)].value + ' UTC')
        if sheet[schema['columns']['actual_development_time_finish'] + str(i)].value is not None:
            project['finish'] = str_to_rfc(sheet[schema['columns']['actual_development_time_finish'] + str(i)].value + ' UTC')

        for key in schema['columns']:
            if key not in exc_lst and sheet[schema['columns'][key] + str(i)].value is not None:
                project[key] = sheet[schema['columns'][key] + str(i)].value


        project_planned = {
            "id": project['id'],
            "wbs": project['wbs'],
            "parent_id": project['parent_id']
        }

        if sheet[schema['columns']['estimated_development_time_summary'] + str(i)].value is not None:
            project_planned['worktime'] = str(sheet[schema['columns']['estimated_development_time_summary'] + str(i)].value) + 'h'


        rdf.add_project(**project)
        rdf.add_project_baseline_planned(**project_planned)

    else:
        print("ERROR: It is not WBS structure or task does not have a name: " + sheet[schema['columns']['wbs'] + str(i)].value + " " + sheet[schema['columns']['name'] + str(i)].value)

    i = i + 1

print("!!! FINISHED !!!")
    



!!! FINISHED !!!


In [12]:
import pydgraph


url = 'localhost:9080'

client_stub = pydgraph.DgraphClientStub(url)
client = pydgraph.DgraphClient(client_stub)

txn = client.txn()

try:
    txn.mutate(set_nquads=str(rdf))
    txn.commit()
except pydgraph.AbortedError:
    # Retry or handle exception.
    txn.discard()
    print(pydgraph.AbortedError)
finally:
    txn.discard()
    print('Data sent successfully.')

Data sent successfully.
